In [16]:
import requests               
import pandas as pd          
from scrapy import Selector 
import pprint 
from tqdm import tqdm

In [ ]:
bbclink = 'https://www.bbcgoodfood.com/search?q='
response = requests.get(bbclink)
sel = Selector(text=response.text)
links = []

recipecards = sel.css('main div.search-results div.card__section.card__content a ::attr(href)').getall()
links.extend(recipecards)  
prefix = "https://www.bbcgoodfood.com/recipes/"
final_links = [prefix + item for item in links] 
print(final_links)

In [17]:
base_url = 'https://www.bbcgoodfood.com/search?q=&limit=1000&page='

links = []
for i in tqdm(range(1, 11)):
    response = requests.get(base_url + str(i))
    sel = Selector(text = response.text)
    recipecards = sel.css('main div.search-results div.card__section.card__content a ::attr(href)').getall()
    links.extend(recipecards)  

prefix = "https://www.bbcgoodfood.com/recipes/"
final_links = [prefix + item for item in links] 

len(final_links)

100%|██████████| 10/10 [01:33<00:00,  9.38s/it]


10011

### Create a DataFrame containing the nutritional data of all the recipes on BBCGoodFood 

In [45]:
session = requests.Session()

def get_nutrition(url):
    response2 = session.get(url)
    sel2 = Selector(text=response2.text)
    table = sel2.css('table.key-value-blocks.hidden-print.mt-xxs')
    bbc = {}
    bbc['name'] = sel2.css('h1.heading-1 ::text').get()
    bbc['calories'] = table.css('td.key-value-blocks__value ::text').get()
    bbc['salt'] = table.xpath('.//*[contains(text(), "salt")]/..').css('td.key-value-blocks__value ::text').get()
    bbc['fat'] = table.xpath('.//*[contains(text(), "fat")]/..').css('td.key-value-blocks__value ::text').get()
    return bbc

bbc = [get_nutrition(url) for url in tqdm(final_links)]

df_bbc = pd.DataFrame(bbc)
df_bbc.head()

100%|██████████| 10011/10011 [2:44:43<00:00,  1.01it/s]


,name,calories,salt,fat
0,Chicken & chorizo jambalaya,445,1.2,10
1,Lemon drizzle cake,399,0.3,21
2,Chilli con carne recipe,387,2.32,17
3,Best ever chocolate brownies recipe,150,0.1,9
4,Creamy courgette lasagne,405,1.36,21


### Save the data as a CSV file 

In [47]:
df_bbc.to_csv('../data/bbc_data.csv', index=False)

In [3]:
title1 = "Creamy Lemon-Basil Chicken Pasta"
title2 = "Chicken & bacon pasta"

from sentence_transformers import SentenceTransformer
sentences = [title1, title2]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[-0.02184611  0.00591048  0.00187744 ... -0.00569096 -0.02545492
  -0.01958655]
 [-0.01549129  0.00366342  0.00322168 ...  0.0223089  -0.02564218
  -0.01029447]]


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
embedding1 = embeddings[0].reshape(1, -1)
embedding2 = embeddings[1].reshape(1, -1)

# Calculate cosine similarity
similarity_score = cosine_similarity(embedding1, embedding2)[0][0]
print(f"Cosine Similarity: {similarity_score}")

Cosine Similarity: 0.6690070629119873
